# Simulation of a cart pendulum

After the theory, it's time to simulate the card-pendulum model using state-variables in feedback control. First of all via Matlab it's explored the characteristic of the model like poles, F-matrix, controllability and observability. Then, a simulation of Linear Quadratic Regulator and Linea Quadratic Gaussian in particular with noise and disturbance are used. For the second one, it's inserted an input step reference to show the response of state-variables, in particular $x$ and $\theta$; it's also use Simulink, a Matlab tool (refer <cite data-cite="grewal2014kalman"></cite> for insight) for a better comprehension. Finally an input reference to track it is used in a feedback control system. A simple gain and an integral control are used to track an input step as last simulations.

## The model
First of all it's worthwhile to design a non linear function of the cart-pendulum derived from $\eqref{eq:set-diff-eq}$.
Then, it's used the m-code to check a situation when the pendulum is in the "up" position (unstable equilibrium) and the initial condition is $y[t=0] = [x=0;\dot x=0;\theta=pi;\dot \theta=.5]$. It would be produced a periodic oscillation of $\theta$ because acceleration of pendulum put it out of its position of equilibrium (unstable). Think the cart pendulum over a railway and the pendulum-leg can rotate over its up down position freely without falling down a physical ground. Multi-oscillation are dumped for the cart-friction represented by a "d" parameter.

So the function cartpend define the unlinear differential equations:
```
function dy = cartpend(y,m,M,L,g,d,u)

Sy = sin(y(3));
Cy = cos(y(3));
D = m*L*L*(M+m*(1-Cy^2));

dy(1,1) = y(2);
dy(2,1) = (1/D)*(-m^2*L^2*g*Cy*Sy + m*L^2*(m*L*y(4)^2*Sy - d*y(2))) + 
m*L*L*(1/D)*u;
dy(3,1) = y(4);
dy(4,1) = (1/D)*((m+M)*m*g*L*Sy - m*L*Cy*(m*L*y(4)^2*Sy - d*y(2))) - 
m*L*Cy*(1/D)*u +.01*randn;
```
And the rest of the code to plot the variables evolution

```
m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

tspan = 0:.1:30;
% pendulum in up positoin omega = .5
y0 = [0; 0; pi; .5];
% solve ode
[t,y] = ode45(@(t,y)cartpend(y,m,M,L,g,d,0),tspan,y0);
% plot theta(t)
plot(t,y(:,3))
```

|          |     
|:----------:|
|![thetapen\label{thetapend}](images/thetapend.png)|

<p style="text-alignment=left;"><a name="fig11">Fig. 11</a> - Imposing boundary condition to put the pendulum out of its unstable equilibrium and showing the evolution </p>

## The modal form and the meaning of its eigenvalues

Like shown in the <a href="#section_1_2">chapter 2.2</a> the modal matrices form is really useful to shown poles over the its diagonal. Matlab gives the possibility to transform the initial set into the desired modal by the command *canon*. T is the transformation between these two representations that using different state variables describe the same physical system. The m-code:

```
s = 1; % pendulum up (s=1)
%% Base system Matrices
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];
H = [1 1 1 1];
J = 0;
sys = ss(F,G,H,J);
[modal_sys,T] = canon(sys, 'modal')

modal_sys =
 
  A = 
            x1       x2       x3       x4
   x1        0        0        0        0
   x2        0    2.434        0        0
   x3        0        0   -2.467        0
   x4        0        0        0  -0.1665
 
  B = 
            u1
   x1        1
   x2   0.2424
   x3  -0.2807
   x4   -1.052
 
  C = 
              x1         x2         x3         x4
   y1          1    0.02457    0.02786     0.9486
   y2          0    0.05981   -0.06874    -0.1579
   y3          0    0.07876    0.07793  -0.002645
   y4          0     0.1917    -0.1923  0.0004404
 
  D = 
       u1
   y1   0
   y2   0
   y3   0
   y4   0
 
Continuous-time state-space model.


T =

    1.0000    6.0000   -0.0000   -2.0000
         0   -0.0996    6.3858    2.6237
         0   -0.1138    6.3656   -2.5799
         0   -6.3195   -0.3524    2.1163
```
A note about $\pmb A$ matrix; it's known that poles on the right half-plane means unstable systems. It's a confirmation of instability when the pendulum is in the top position (look at the <a href="#fig11">figure 11</a> $\theta$ when the pendulum comes out of its unstable position i.e. either few cents of degree off its top position or like the simulation $\omega = 0.5$) 

It turns out that the transformation matrix that will convert the state description matrices to modal form has its columns the eigenvectors of $F$. In order to get the eigenvalues of $F$ is also possible to use the MATLAB function "eig()" like shown below:
```
>> eig(F)

ans =

         0
   -2.4674
   -0.1665
    2.4339
```
Backing to $\eqref{eq:C}$ it's time to check the controllability of the card pendulum system:
```
>> ctrb(A,B)

ans =

         0    0.2000   -0.0400    0.2080
    0.2000   -0.0400    0.2080   -0.0816
         0    0.1000   -0.0200    0.6040
    0.1000   -0.0200    0.6040   -0.1408

>> rank(ctrb(A,B))

ans =

     4
```
The rank of $\mathcal C$ is full and it's possible to control the **full state system** by looping **x**.  

## Hot to move poles to control the cart-pendulum
Matlab uses *place* command in order to move poles location of the plant. The first code snippet shows some set of pole location and plot the x-cart position. It can be observed than the more to the left position the poles are the more aggressive response the plant will have.

```
clear all, close all, clc

m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = 1; % pendulum up (s=1)

A = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

B = [0; 1/M; 0; s*1/(M*L)];
eig(A)

rank(ctrb(A,B))  % is it controllable

%%  Pole placement
% For more complex case a more reliable formula is available with 
% function place
% p is a vector of desired eigenvalues
p0 = [-.01; -.02; -.03; -.04]; % not enough
p1 = [-.3; -.4; -.5; -.6];  % just barely
p2 = [-1; -1.1; -1.2; -1.3]; % good
p3 = [-2; -2.1; -2.2; -2.3]; % aggressive
p4 = [-3; -3.1; -3.2; -3.3]; % aggressive
%p = [-3.5; -3.6; -3.7; -3.8]; % breaks
% create a list of vectors
list_p = {p1,p2,p3,p4};
% K = lqr(A,B,Q,R);

% Plot a list of non-minimum phase zero 
for k = 1:numel(list_p)
    tspan = 0:.001:30;
    vec_p = list_p{k};
    % Used place instead acker function
    K = place(A,B,vec_p)
    y0 = [-3; 0; pi+.1; 0];
    [t,y] = ode45(@(t,y)cartpend(y,m,M,L,g,d,-K*(y-[1; 0; pi; 0])),tspan,y0);    
    plot(t,y(:,1)); hold on;
end
```
Print out the K values:
```
K =

   -0.0360   -1.3420   71.9720   18.6840


K =

   -1.7160   -7.0260  142.5320   58.0520


K =

  -21.2520  -40.6460  379.6040  165.2920


K =

  -98.2080 -125.8660  851.5160  375.7320
```

|          |
|:----------:|
|![modalblock\label{modalblock}](images/poleplacement.png)|

<p style="text-alignment=left;"><a name="fig12">Fig. 12</a> - Some $K$ values based on different pole placement. The start position is x = -3 and $\theta= \pi +0.1$. Finale reference position is x = 1 $\theta = \pi$. It's worthwhile to note than more aggressively is equal to require more energy and less robustness </p>

It should be to note the system has to work harder to move pole long way (large gains) and reduce the control effort. In addition it's necessary to mention that the more aggressive will be the control the less robust the system will be because non linearity comes out respect the approximation done.

### The Linear Quadratic Regulator
A best trade-off from control effort and fast response is computed using the Linear Quadratic Regulator. In Matlab, the $K_r$ choice is obtained via command:
```
>> Kr = lqr(F,G,Q,R);
``` 
Some plots are added to show *the spectrum* of state-variables. Another graph is added to plot the functionals represented the best trade-off versus different values of $K$

```
%% Compare with many examples of Pole Placement
x0 = [-1; 0; pi+.1; 0];  % initial condition 
wr = [1; 0; pi; 0];      % reference position
K = lqr(F,G,Q,R);
u=@(x)-K*(x - wr);       % control law
[t,x] = ode45(@(t,x)pendcart(x,m,M,L,g,d,u(x)),tspan,x0);
xLQR = x;
for k=1:length(t)
    JLQR(k) = (x(k,:)-wr')*Q*(x(k,:)'-wr) + u(x(k,:)')^2*R;
end

CC = [0    0.4470    0.7410
    0.8500    0.3250    0.0980
    0.9290    0.6940    0.1250
    0.4940    0.1840    0.5560
    0.4660    0.6740    0.1880
    0.3010    0.7450    0.9330
    0.6350    0.0780    0.1840];

CCgray = [0.2    0.6470    0.9410
    0.9500    0.4250    0.1980
    1    0.7940    0.2250
    0.5940    0.2840    0.6560
    0.4660    0.6740    0.1880
    0.3010    0.7450    0.9330
    0.6350    0.0780    0.1840];

%%try 100 different poles choises, plot graph of state-variables
%%and cost-functions together LQR gain.

for count = 1:100
    p = [-.5-3*rand; -.5-3*rand; -.5-3*rand; -.5-3*rand];
	K = place(F,G,p);
    u=@(x)-K*(x - wr);       % control law
    [t,x] = ode45(@(t,x)pendcart(x,m,M,L,g,d,u(x)),tspan,x0);
    figure(1)
    for j=1:4
        plot(t(1:50:end),x(1:50:end,j),'Color',[.5 .5 .5] + .3*CC(j,:)), hold on;
    end
    for k=1:length(t)
        J(k) = (x(k,:)-wr')*Q*(x(k,:)'-wr) + u(x(k,:)')^2*R;
    end
    figure(2)
    Jz = cumtrapz(t,J);
    plot(t(1:50:end),Jz(1:50:end),'Color',[.5 .5 .5]), hold on;
end
figure(1)
    for j=1:4
        plot(t(1:10:end),xLQR(1:10:end,j),'Color',CC(j,:),'LineWidth',2)
        l1 = legend('x','v','\theta','\omega')
    end
figure(2)
plot(t,cumtrapz(t,JLQR),'k','LineWidth',2)
```

|           |
|-----------|
|![fig1.4](images/poleplacement2.png)|
|![fig1.4.2](images/costfunctions.png)|

<p><a name="Fig13"> Fig. 13 </a>- Solution for Linear Quadratic Regulator compared with different set of poles locations; then the cost function is plotted in a visual way to show its functional value </p>

## The observability of the cart-pendulum
This code snippet goes to calculate the observability matrix when the measurement matrix **H** (in the code is C) select a sensor related to x cart position and then reply the calculation for $\theta$ measurement. It can be highlighted that the determinant is nonzero (full rank) in the first trial and zero in the second one. In particular  tuns out that in the second is impossible to reconstruct the x position because that coordinate does not affect the others; equations of cart pendulum are invariant for translations in x position. It's important reconstruct the cart position to have full estimate of state variable putting them in the LQR block for resulting compensator placed in the feed back (as it will see later).

```
clear all, close all, clc

m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = -1; % pendulum up (s=1)

A = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

B = [0; 1/M; 0; s*1/(M*L)];

C = [1 0 0 0]; 

obsv(A,C)
det(obsv(A,C))


C = [0 0 1 0]; %%  only observable if x measured... because x can't be reconstructed

obsv(A,C)
det(obsv(A,C))
```
The terminal result:
```
ans =

    1.0000         0         0         0
         0    1.0000         0         0
         0   -0.2000    2.0000         0
         0    0.0400   -0.4000    2.0000


ans =

     4


ans =

         0         0    1.0000         0
         0         0         0    1.0000
         0    0.1000   -6.0000         0
         0   -0.0200    0.2000   -6.0000


ans =

     0
```
---

###  Simulation of plant disturbance and noise measurement. 
To give a demonstration of best choice of $L \equiv K_f$ it's considered a system with $J = 0$ in $\eqref{eq:output}$ and pendulum in bottom position. It's better create a practical solution around a stable equilibrium point because the algebra is the same. The system represented in the <a href="#fig14">picture</a> shows the linearized system with input $u$ to be the force applied to the cart pendulum, $w_d$ the gaussian zero-mean disturbance applied to the model and the cart position $(\pmb H = [1 0 0 0])$ noised as output of state variable observed. The goal is reconstruct the full state estimate. However the strategy is reconstruct the normal form of the first order state-equation. To do it is necessary to represent the system like 2 *augmented* blocks to feed in Matlab functions: 

\begin{align}
\pmb Q &= 0.1 \pmb{I} & \pmb x = \begin{bmatrix} x\\ \dot x\\ \theta\\      \dot{\theta} \end{bmatrix}\\
R &= 1   
\end{align}

\begin{equation} \label{eq:aug-eqs}
\begin{split}
\pmb{G}_A & = \begin{bmatrix}\pmb G & \pmb Q & \pmb{\underline{0}}x \pmb B\end{bmatrix}\\
\pmb{F}_A & = \pmb{F}\\
\pmb{H}_A & = \pmb H\\
\pmb{J}_A & = \begin{bmatrix} 0 & 0 & 0 & 0 & 0 & R \end{bmatrix}
\end{split}
\end{equation}

|       | 
|:-------:|
|![fig5](images/kfscheme.png)|

<p><a name="fig14">Fig 14</a> A scheme of Kalman Filter; d and n are disturbance of state system and noise measurement introduced in the simulation. Note $y$ true is the output of the plant and $y$ noise the estimator input</p>

According to the equations $\eqref{eq:estimator}$, by inspecting $\eqref{eq:stat-form}$ and $\eqref{eq:output}$ the *new* estimator-system matrices are:

\begin{equation}
\label{eq:kalman-system}
\begin{split}
\pmb{F}_k &= \pmb F - \pmb{K}_f \pmb H\\
\pmb{G}_k &= \begin{bmatrix}\pmb G \pmb{K}_f \end{bmatrix}\\
\pmb{H}_k &= \pmb I\\
\pmb{J}_k &= \pmb{0}
\end{split}
\end{equation}

The input $u$ is augmented also to taking into account of disturbance plant and noise measurement:

\begin{equation}
\label{eq:u_noise}
\pmb{u}_{aug} = \begin{bmatrix} u & {Q}^2 * \pmb{u}_{dist} & u_{noise}\end{bmatrix}
\end{equation}

The code:
```
clear all, close all, clc

m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = -1; % pendulum up (s=1)

% y = [x; dx; theta; dtheta];
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];

H = [1 0 0 0];  

J = zeros(size(H,1),size(G,2));

%%  Augment system with disturbances and noise
Q = .1*eye(4);  % disturbance covariance
R = 1;       % noise covariance

GA = [G Q 0*G];  % augment inputs to include disturbance and noise

sysH = ss(F,GA,H,[0 0 0 0 0 R]);  % build big state space system... with single output

% system with full state output, disturbance, no noise
sysFullOutput = ss(F,GA,eye(4),zeros(4,size(GA,2)));  

%%  Build Kalman filter
[L,P,E] = lqe(F,Q,H,Q,R);  % design Kalman filter
Kf = (lqr(F',H',Q,R))';   % alternatively, possible to design using "LQR" code

sysKF = ss(F-L*H,[G L],eye(4),0*[G L]);  % Kalman filter estimator

%%  Estimate linearized system in "down" position (Gantry crane)
dt = .01;
t = dt:dt:50;

uDIST = randn(4,size(t,2));
uNOISE = randn(size(t));
u = 0*t;
u(100:120) = 100;     % impulse
u(1500:1520) = -100;  % impulse

uAUG = [u; Q*Q*uDIST; uNOISE];

[y,t] = lsim(sysH,uAUG,t);% Here y is the noise measured signal 

[xtrue,t] = lsim(sysFullOutput,uAUG,t); % The x cart-pendulum position of the plant

[x,t] = lsim(sysKF,[u; y'],t); % The estimate x of cart-pendulum


plot(t,xtrue,'-',t,x,'--','LineWidth',2) %% Plot together y,x true,x estimate
figure
plot(t,y)
hold on
plot(t,xtrue(:,1),'r')
plot(t,x(:,1),'k--')
```
---

|         |
|:---------:|
|![graph](images/cartpendxpos.png)|
<p> <a name="simnoise"> Fig. 15 </a> Simulation of a noise position from an impulse system. Estimate position "cover" the true</p>

|         |
|:---------:|
|![graph-est](images/true-estimate.png)|
<p> <a name="true-estimate"> Fig. 16 </a> Showing the state variable "true" and estimate</p>

With the estimate state, it's the time to "*feedback*" it in a Linear Quadratic Regulator to combine control and estimation together. 

### Simulation of LQG in a feed back control. How to put all togheter. 

To apply an LQG regulator to the inverted pendulum on a cart, we will simulate the full nonlinear system in Simulink, as shown in <a href="#LQG">figure 17</a>.
The non-linear cart pendulum is connected by the force $u$ in addiction to a gaussian random noise disturbance that affected the model. The output of the cart-pendulum is the state-variable vector connected with $\pmb H$ matrix to go input in the estimator block only with the position-cart sensor noise measurement gaussian distributed also; the relation is $y = \pmb H \pmb x + w_n$. The Kalman filter reconstruct the full state variable for the compensator block.  

|           |
|:---------:|
|![fig9](images/LQG.png)|

<p><a id="LQG">Fig. 17 </a> Matlab Simulink model for sensor-based LQG feedback control.</p>

The system starts near the vertical equilibrium, at $\pmb x_0 = [0, 0, 3.14, 0]$, and **it's commanded a step** in the cart position from x = 0 to x = 1 at t = 10. The resulting response is shown in <a href="#stateLQG">figure 18</a>. Despite noisy measurements (<a href="#positionLQG">figure 19</a>) and disturbances (<a href="#noiseLQG">figure 20</a>), the controller is able to effectively track the reference cart position while stabilizing the inverted pendulum.

|           |
|:---------:|
|![figure10](images/stateLQG.png)|
<p><a id="stateLQG">Fig. 18</a> Output response using LQG feedback control.</p>

|           |
|:---------:|
|![figure11](images/positionLQG.png)|
<p><a id="positionLQG">Fig. 19</a> Noisy measurement used for the Kalman filter, along with the underlying noiseless signal and the Kalman filter estimate.</p>

|           |
|:---------:|
|![figure12](images/noiseLQG.png)|
<p><a id="noiseLQG">Fig. 20</a> Noisy measurement used for the Kalman filter, along with the underlying noiseless signal and the Kalman filter estimate</p>

The m-code set some values needed for the simulation
```
clear all, close all, clc

%% Set cart-pendulum parameters
m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = 1; % pendulum up (s=1)
%% Base system Matrices
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];

% Set cart-position
H = [1 0 0 0];  

J = zeros(size(H,1),size(G,2));

%% Linear Quadratic Regulator
Q = [1 0 0 0;
    0 1 0 0;
    0 0 1 0;
    0 0 0 1];
R = .000001;
K = lqr(F,G,Q,R);  % design controller u = -K*x


%% Augmented system
%  Augment system with disturbances and noise
Vdmag = .04;
Vd = Vdmag*eye(4);  % disturbance covariance
Vn = .0002;       % noise covariance

% GF sysC sysFulloutput not used in Simulink
GF = [G Vd 0*G];  % augment inputs to include disturbance and noise
sysH = ss(F,GF,H,[0 0 0 0 0 Vn]);  % build big state space system... with single output
 % system with full state output, disturbance, no noise
sysFullOutput = ss(F,GF,eye(4),zeros(4,size(GF,2))); 

%  Build Kalman filter 2 alternatives
[L,P,E] = lqe(F,eye(4),H,Vd,Vn);  % design Kalman filter
Kf = (lqr(F',H',Vd,Vn))';   % alternatively, possible to design using "LQR" code

% The system matrices for LQE. Used in Simulink
sysKF = ss(F-Kf*H,[G Kf],eye(4),0*[G Kf]);  % Kalman filter estimator

```

Here the m-code to print the result
```
CC = [0    0.4470    0.7410
    0.8500    0.3250    0.0980
    0.9290    0.6940    0.1250
    0.4940    0.1840    0.5560
    0.4660    0.6740    0.1880
    0.3010    0.7450    0.9330
    0.6350    0.0780    0.1840];

figure(1)
plot(yn.Time,yn.Data(:,2),'Color',[.5 .5 .5])
hold on
plot(x.Time,x.Data(:,1),'Color',[0 0 0],'LineWidth',2)
plot(yKF.Time,yKF.Data(:,1),'--','Color',CC(1,:),'LineWidth',2)
set(gcf,'Position',[100 100 500 200])
xlabel('Time')
ylabel('Measurement')
l1 = legend('y (measured)','y (no noise)','y (KF estimate)');
set(l1,'Location','SouthEast')
grid on

%%
figure(2)
subplot(3,1,1)
plot(wd.Time(1:100:end),wd.Data(1:100:end,1),'LineWidth',1.2)
xlim([45 50])
ylabel('Disturbance, d')
subplot(3,1,2)
plot(wn.Time(1:100:end),wn.Data(1:100:end,1),'LineWidth',1.2)
xlim([45 50])
ylabel('Noise measurement, n')
subplot(3,1,3)
plot(yn.Time(1:100:end),yn.Data(1:100:end,1),'LineWidth',1.2)
xlim([45 50])
set(gcf,'Position',[100 100 500 200])
xlabel('Time')
ylabel('Actuation, u')
grid on

%%
figure(3)
plot(x.Time,x.Data,'LineWidth',2)
l1 = legend('x','v','\theta','\omega')
set(gcf,'Position',[100 100 500 200])
xlabel('Time')
ylabel('State')
grid on

```

## A simulation for a cart pendulum into a feed-back control with a reference input
<a name="section_4_2"></a>
Having described in chapter 5 how to obtain precise result it's shown below the block scheme *without* a gain reference. It's expected to obtain an output error.

|                        |
|:----------------------:|
|![HL1.png](images/HL1.png)|

<p style="text-align:center"><a name="fig21">Fig. 21 </a> Block scheme of reference system with Linear Quadratic Regulator</p>

The design criteria for this system with the cart receiving a 0.2 step input are as follows:

 - Settling time for x and theta of less than 5 seconds.
 - Rise time for x of less than 1 second
 - Overshoot of $\theta$ less than 20 degree (0.35 radians). Please note in this lab the up position is 0 degree and down position is $\pi$
 - Steady-state error within 2%

For an inverted pendulum it is unrealistic to consider just the single output system. Using state-space/state variables methods it is relatively simple to work with a multi-output system, so in this hand-on lab it will design a controller with both the pendulum angle and the cart position in mind.

A piece of code
```
%% Set cart-pendulum parameters
m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = 1; % pendulum up (s=1)
%% Base system Matrices
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];

% Set cart-position
H = [1 0 0 0; 
        0 0 1 0];  

J = zeros(size(H,1),size(G,2))
```

The next step in the design process is to assume to have full-state feedback (i.e. that we can measure all four states), and find the vector $K$ which determines the feedback control law. This can be done in a number of ways. If it's known the desired closed-loop poles, it's possible to use the *place* or *acker* command. Another option is to use the *lqr* function; this will give you the optimal controller (under certain assumptions. The lqr function allows to choose two parameters, R and Q, which will balance the relative importance of the input and state in the cost function to optimize the trade-off. The simplest case is to assume R=1, and Q=H'*H. It's worthwhile to note that it's possible using both outputs (the pendulum's angle and the cart's position). Essentially, the lqr method allows to control both outputs. In this case, it is pretty easy to do. The controller can be tuned by changing the nonzero elements in the Q matrix to get a desirable response.
To find the structure of Q it's possible use the matlab command:
```
>> H'*H

ans =

     1     0     0     0
     0     0     0     0
     0     0     1     0
     0     0     0     0
```
The element in the 1,1 position will be used to weight the cart's position and the element in the 3,3 position will be used to weight the pendulum's angle. The input weighting R will remain at 1. Now that we know what the Q matrix should look like we can experiment to find the K matrix that will give a good controller. Then it's the time to find the K matrix and plot the response all in one step so that changes can be made in the control and be seen automatically in the response. Enter the following text into your m-file: 
```
%% Linear Quadratic Regulator
%% try different value of Q
Q = [1 0 0 0;
    0 0 0 0;
    0 0 1 0;
    0 0 0 0];
R = 1;
K = lqr(F,G,Q,R);  % design controller u = -K*x
Ac = [(F-G*K)];
Bc = [G];
Cc = [H];
Dc = [J];

T=0:0.01:5;
U=0.2*ones(size(T));
[Y,X]=lsim(Ac,Bc,Cc,Dc,U,T);
plot(T,Y)
legend('Cart','Pendulum')
```
```
>> K

K =

   -1.0000   -5.4325  153.2343   63.8985
```

|                                                    |
|:--------------------------------------------------:|
|![1stepresponse.png](images/1stepresponse.png)      |

<p style="text-align:left"><a name="fig22">Fig. 22 </a> The curve in red represents the pendulum's angle, in radians and the curve in blue represents the cart's position in meters. As you can see, this plot is not satisfactory</p>

As you can see, this plot is not satisfactory. The pendulum and cart's overshoot appear fine, but their settling times need improvement and the cart's rise time needs to go down. The cart is not near the desired location but has in fact moved in the other direction. This error will be dealt in the next section and right now the focus is on the settling and rise times. Increasing $Q_x$ makes the settling and rise times go down, and lowers the angle the pendulum moves. Using $Q_x=5000$ and $Q_{\theta}=100$, the following value of K and step response were found: 
```
%% Linear Quadratic Regulator
%% try different value of Q
Q = [5000 0 0 0;
    0 0 0 0;
    0 0 100 0;
    0 0 0 0];
R = 1;
K = lqr(F,G,Q,R);  % design controller u = -K*x
Ac = [(F-G*K)];
Bc = [G];
Cc = [H];
Dc = [J];

T=0:0.01:5;
U=0.2*ones(size(T));
[Y,X]=lsim(Ac,Bc,Cc,Dc,U,T);
plot(T,Y)
legend('Cart','Pendulum')

>> K

K =

  -70.7107  -91.3062  636.5903  280.1483
```

|                                            |
|:------------------------------------------:|
|![2stepresponse.png](images/2stepresponse.png)|

<p style="text-align:left"><a name="fig23">Fig. 23 </a> Raising $Q_x$ and $Q_{\theta}$ imply more control effort and smaller tracking error</p>

If it's increased $Q_x$ and $Q_{\theta}$ even higher, it's possible to improve the response even more. This plot was chosen because it satisfied the design requirements while keeping x and y as small as possible. In this problem, $Q_x$ and $Q_{\theta}$ have been used to describe the relative weight of the tracking error in the cart's position and pendulum's angle versus the control effort. The higher these 2 parameters are, the more control effort is used, but the smaller the tracking error. The system response has a settling time under 3 seconds.

 In contrast to the other design methods, where it's feedback the output and compare it to the reference input to compute an error, with a full-state feedback controller it's feeding back all the states. It necessary to compute what the steady-state value of the states should be, multiply that by the chosen gain $K$, and use a new value as our reference for computing the input. This can be done by adding a constant gain $\bar{N}$ after the reference. The schematic below shows this relationship: 

|             |
|:-----------:|
|![systemwithreference2.png](images/systemwithreference2.png)|

<p style="text-align:center"><a name="fig22">Fig. 22 </a> Adding a gain in a reference input</p>

Nbar can be found using the user-defined function rscale. A different H had to be used because the rscale function will not work for multiple outputs. However, the Nbar found is correct, as you can see from the output below:

|         |
|:-------:|
|![3stepinput.png](images/3stepinput.png)|

<p style="text-align:center"><a name="fig23">Fig. 23 </a> Step response with LQR and $\bar{N}$ control</p>

Here the matlab code:
```
%% Set augmented system
Ac = [(F - G*K)];
Bc = [G];
Cc = [H];
Dc = [J];

%% Plot system with reference
T=0:0.01:25;
U=0.2*ones(size(T));
[Y,X]=lsim(Ac,Bc,Cc,Dc,U,T);
%plot(T,Y)
%legend('Cart','Pendulum')


%% Set gain reference
Cn = [1 0 0 0];
Nbar = rscale(F,G,Cn,0,K);
Bcn = [Nbar*G];
[Y,X] = lsim(Ac,Bcn,Cc,Dc,U,T);
plot(T,Y)
legend('Cart position','pendulum angle')
```
```
function[Nbar]=rscale(A,B,C,D,K)

s = size(A,1);
Z = [zeros([1,s]) 1];
N = inv([A,B;C,D])*Z';
Nx = N(1:s);
Nu = N(1+s);
Nbar=Nu + K*Nx;
```
Note that rscale function implements the equation $\eqref{eq:reference-input2}$ and $\eqref{eq:reference-input1}$

## Add an integrator in a feedback control for inverted pendulum. The state-space approach
<a name="section_4_4"></a>
For the system:
\begin{align}\label{eq:state-eq-control-integral}
\pmb{\dot x} &= \pmb F \pmb x + \pmb G u + \pmb G_1 w \\
y &= \pmb H \pmb x \nonumber
\end{align}

it's possible feed back the integral control, $e=y-r$, together the state of the plant $\pmb x$, by augmenting the plant state with the extra integral state $x_I$ which respond to the differential equation:
\begin{equation}\label{eq:integral-error}
\dot{x_I} = \pmb H \pmb x - r(=e), x_I = \int_{}^{t}{e \space d\tau}
\end{equation}

The augmented state equations become:
\begin{equation}\label{eq:aug-state-eq}
\begin{bmatrix}
\dot x_I\\
\pmb{\dot x}
\end{bmatrix}=
\begin{bmatrix}
0 & \pmb H\\
\mathbf{0} & \pmb F
\end{bmatrix}
\begin{bmatrix}
x_I\\
\pmb x
\end{bmatrix}+
\begin{bmatrix}
0\\
\pmb G
\end{bmatrix}u -
\begin{bmatrix}
1\\
\mathbf{0}
\end{bmatrix} r +
\begin{bmatrix}
0\\
\pmb{G}_1
\end{bmatrix} w
\end{equation}

and the feedback law is:
\begin{equation}\label{eq:feedback-law}
u = - \begin{bmatrix}
K_I & \pmb K_0
\end{bmatrix}
\begin{bmatrix}
x_I\\
\pmb x
\end{bmatrix}
\end{equation}

The problem is traced back to a LQR system.

|           |
|:---------:|
|![simulink-integral-control.png](images/simulink-integral-control.png)|

<p style="text-align:center"><a name="fig25">Fig. 25 </a> Simulink for integral control with reference</p>

Building a simulation via *simulink* means recover the LQR part plus the feedback with reference, an integral control plus a random number generator to create the disturbance over the *u* line. In particular the system is under a unit step reference that cart pendulum position "x" has to follow.
The matlab code follow the LQR part for the augmented system:
```
clear all, close all, clc

%% Set cart-pendulum parameters
m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = 1; % pendulum up (s=1)
%% Base system Matrices
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];

% Set cart-position and pendulum angle
H = [1 0 0 0]
        

J = zeros(size(H,1),size(G,2));

%% Integral control augmented system
Fa = [0 H;0 F(1,:);0 F(2,:);0 F(3,:);0 F(4,:)];
Ga = [0; G];
R = 0.0001;
K = lqr(Fa,Ga,eye(5),R);
```
printing K via command line

```
>> K

K =

   1.0e+03 *

   -0.1000   -0.2696   -0.3145    2.0341    0.9096
```

The K values are inserted in the Simulink block.

Here the graphs of simulation:

|         |
|:-------:|
|![integral-control-with-reference.png](images/integral-control-with-reference.png)|

<p style="text-align:left"><a name="fig26">Fig. 26 </a>As noted $\theta=\pi$ when pendulum is in "up" unstable position. The feed back reject the disturbance and the cart tracks the step reference starting at time $t=10$ seconds </p>